<a href="https://colab.research.google.com/github/mahmoodtt1/OR/blob/main/Signal_Coordination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from pyomo.environ import *

# install needed libraries

!pip install networkx matplotlib

# install pyomo solvers.

!apt-get install -y glpk-utils

!apt-get install -y coinor-cbc


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
glpk-utils is already the newest version (5.0-1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
coinor-cbc is already the newest version (2.10.7+ds1-1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [22]:
from pyomo.environ import *

# Create model
model = ConcreteModel()

# Sets
model.I = Set(initialize=['1'])  # single coordinated phase for now

# Parameters
C = 120
t_AB = 65
g_min = 15
g_max = 60
M = 1000  # Large constant for Big-M

model.C = Param(initialize=C)
model.t_AB = Param(initialize=t_AB)
model.g_min = Param(initialize=g_min)
model.g_max = Param(initialize=g_max)
model.M = Param(initialize=M)

# Variables
model.s_A = Var(model.I, bounds=(0, C))                     # start at A
model.s_B = Var(model.I, bounds=(0, C))                     # start at B
model.g_A = Var(model.I, bounds=(g_min, g_max))             # green time at A
model.g_B = Var(model.I, bounds=(g_min, g_max))             # green time at B

model.e_A = Var(model.I)                                    # end time at A
model.e_B = Var(model.I)                                    # end time at B

model.early_end = Var(model.I)                              # min(e_A, e_B - t_AB)
model.late_start = Var(model.I)                             # max(s_A, s_B - t_AB)
model.o = Var(model.I, within=NonNegativeReals)             # overlap duration

model.x = Var(model.I, within=Binary)                       # for min()
model.y = Var(model.I, within=Binary)                       # for max()

# Constraints

# End time definitions
def end_A_rule(m, i):
    return m.e_A[i] == m.s_A[i] + m.g_A[i]
model.end_A = Constraint(model.I, rule=end_A_rule)

def end_B_rule(m, i):
    return m.e_B[i] == m.s_B[i] + m.g_B[i]
model.end_B = Constraint(model.I, rule=end_B_rule)

# Shifted values (in constraints directly, so no extra variables)

# Min(e_A, e_B - t_AB) using binary x
def min_upper_1(m, i):
    return m.early_end[i] <= m.e_A[i]
def min_upper_2(m, i):
    return m.early_end[i] <= m.e_B[i] - m.t_AB
def min_lower_1(m, i):
    return m.early_end[i] >= m.e_A[i] - m.M * (1 - m.x[i])
def min_lower_2(m, i):
    return m.early_end[i] >= m.e_B[i] - m.t_AB - m.M * m.x[i]

model.min1_u1 = Constraint(model.I, rule=min_upper_1)
model.min1_u2 = Constraint(model.I, rule=min_upper_2)
model.min1_l1 = Constraint(model.I, rule=min_lower_1)
model.min1_l2 = Constraint(model.I, rule=min_lower_2)

# Max(s_A, s_B - t_AB) using binary y
def max_lower_1(m, i):
    return m.late_start[i] >= m.s_A[i]
def max_lower_2(m, i):
    return m.late_start[i] >= m.s_B[i] - m.t_AB
def max_upper_1(m, i):
    return m.late_start[i] <= m.s_A[i] + m.M * (1 - m.y[i])
def max_upper_2(m, i):
    return m.late_start[i] <= m.s_B[i] - m.t_AB + m.M * m.y[i]

model.max1_l1 = Constraint(model.I, rule=max_lower_1)
model.max1_l2 = Constraint(model.I, rule=max_lower_2)
model.max1_u1 = Constraint(model.I, rule=max_upper_1)
model.max1_u2 = Constraint(model.I, rule=max_upper_2)

# Overlap definition
def overlap_rule(m, i):
    return m.o[i] == m.early_end[i] - m.late_start[i]
model.overlap_def = Constraint(model.I, rule=overlap_rule)

# Ensure non-negative overlap
def overlap_nonneg(m, i):
    return m.o[i] >= 0
model.overlap_nonneg = Constraint(model.I, rule=overlap_nonneg)

# Objective: maximize total overlap
def objective_rule(m):
    return sum(m.o[i] for i in m.I)
model.obj = Objective(rule=objective_rule, sense=maximize)


In [23]:
solver = SolverFactory("glpk", executable="/usr/bin/glpsol")

results = solver.solve(model, tee=True)



GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmp3ii1z2ft.glpk.raw --wglp /tmp/tmphykd0z2t.glpk.glp --cpxlp
 /tmp/tmp2fcl04w0.pyomo.lp
Reading problem data from '/tmp/tmp2fcl04w0.pyomo.lp'...
/tmp/tmp2fcl04w0.pyomo.lp:88: warning: lower bound of variable 'x15' redefined
/tmp/tmp2fcl04w0.pyomo.lp:88: warning: upper bound of variable 'x15' redefined
12 rows, 11 columns, 30 non-zeros
2 integer variables, all of which are binary
90 lines were read
Writing problem data to '/tmp/tmphykd0z2t.glpk.glp'...
75 lines were written
GLPK Integer Optimizer 5.0
12 rows, 11 columns, 30 non-zeros
2 integer variables, all of which are binary
Preprocessing...
4 constraint coefficient(s) were reduced
11 rows, 8 columns, 26 non-zeros
2 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.800e+02  ratio =  1.800e+02
GM: min|aij| =  8.940e-01  max|aij| =  1.119e+00  ratio =  1.251e+00
EQ: min|aij| =  7.993e-01  max|aij| =  1.

In [24]:

for i in model.I:
    print(f"Phase {i}:")
    print(f"  s_A = {value(model.s_A[i]):.2f}")
    print(f"  g_A = {value(model.g_A[i]):.2f}")
    print(f"  s_B = {value(model.s_B[i]):.2f}")
    print(f"  g_B = {value(model.g_B[i]):.2f}")
    print(f"  Overlap = {value(model.o[i]):.2f}")


Phase 1:
  s_A = 0.00
  g_A = 60.00
  s_B = 65.00
  g_B = 60.00
  Overlap = 60.00
